<a href="https://colab.research.google.com/github/YardenGoraly/Mujoco_fun/blob/main/MuJoCo_fun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Initial setup, you shouldn't have to modify this code

%pip install mujoco
!pip install mediapy

import platform
import os
import subprocess
import mediapy as media

# Detect the operating system and configure GPU rendering accordingly.
if platform.system() == "Linux":
    # Assume Nvidia GPU is present.
    if subprocess.run("nvidia-smi", shell=True).returncode != 0:
        raise RuntimeError(
            "Cannot communicate with GPU. Make sure you are using a GPU runtime."
        )

    # Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
    NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
    if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
        with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
            f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")
    print("Setting environment variable for Nvidia GPU rendering (EGL).")
    os.environ["MUJOCO_GL"] = "egl"

elif platform.system() == "Darwin":
    # Assume running on macOS (Apple Silicon).
    print("Running on macOS. Setting environment variable for GPU rendering using GLFW.")
    os.environ["MUJOCO_GL"] = "glfw"

    media.set_ffmpeg("/opt/homebrew/bin/ffmpeg")
else:
    print("Unsupported platform. GPU rendering might not be configured correctly.")

# Check if MuJoCo installation was successful.
try:
    import mujoco as mj
    mj.MjModel.from_xml_string("<mujoco/>")
except Exception as e:
    raise RuntimeError(
        "Something went wrong during MuJoCo installation. Check the shell output above for more information."
    ) from e

print("MuJoCo installation successful.")

# Other imports and helper functions.
import time
import itertools
import numpy as np
np.set_printoptions(precision=3, suppress=True, linewidth=100)

# Graphics and plotting.
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# On Linux, ensure ffmpeg is installed (this is not applicable on macOS).
if platform.system() == "Linux":
    !command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)

%pip install -q mediapy
import mediapy as media

from IPython.display import clear_output
clear_output()



In [2]:
#More setup 
#More setup
%pip install robot_descriptions
%pip install dm_control

from robot_descriptions import panda_mj_description
from IPython.display import HTML
from robot_descriptions.loaders.mujoco import load_robot_description
from dm_control import mjcf
import dm_control
import PIL.Image


def display_video(frames, framerate=30):
    height, width, _ = frames[0].shape
    dpi = 70
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
      im.set_data(frame)
      return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=False)
    return HTML(anim.to_html5_video())
clear_output()

In [3]:
# Get XML for Sawyer, hand, and ball
ball_xml = """
<mujoco model="ball">
    <worldbody>
        <body name="ball_body" pos="0.8 0.2 0.04">
            <geom name="ball_geom" mass="0.01" friction="1" type="sphere" solref="0.01 0.5" size="0.05" rgba="1 0 0 1"/>
        </body>
    </worldbody>
</mujoco>
"""
hand_path = "mujoco_menagerie/wonik_allegro/right_hand.xml"
sawyer_path = "mujoco_menagerie/rethink_robotics_sawyer/sawyer.xml"

In [4]:
# Define Models
hand_model = mjcf.from_path(hand_path)
sawyer_model = mjcf.from_path(sawyer_path)
ball_model = mjcf.from_xml_string(ball_xml)

# Fingertips in XML are not actually at the tip, so we add a body with an offset
ff_tip = hand_model.find('body', 'ff_tip')
ff_tip.add('body', name='ff_tip_rubber', pos=[0, 0, 0.02])
mf_tip = hand_model.find('body', 'mf_tip')
mf_tip.add('body', name='mf_tip_rubber', pos=[0, 0, 0.02])
rf_tip = hand_model.find('body', 'rf_tip')
rf_tip.add('body', name='rf_tip_rubber', pos=[0, 0, 0.02])
th_tip = hand_model.find('body', 'th_tip')
th_tip.add('body', name='th_tip_rubber', pos=[0, 0, 0.02])

MJCF Element: <body name="th_tip_rubber" pos="0 0 0.02"/>

In [5]:
# Attach hand to the Sawyer
arena = mjcf.RootElement()
sawyer_site = sawyer_model.find('site', 'attachment_site')
attachment_frame = arena.attach(ball_model)
sawyer_site.attach(hand_model)
arena.attach(sawyer_model)

# Set up scene
sky = arena.asset.add('texture', type='skybox', builtin="gradient", rgb1=[0, .2, 1], 
                      rgb2="1 1 1", width=512, height=512)
chequered = arena.asset.add('texture', type='2d', builtin='checker', width=500,
                            height=500, rgb1=[.2, .3, .4], rgb2=[.3, .4, .5])
grid = arena.asset.add('material', name='grid', texture=chequered,
                       texrepeat=[30, 30], reflectance=.1)
arena.worldbody.add('geom', type='plane', size=[10, 10, 10], material=grid)
for x in [-2, 2]:
  arena.worldbody.add('light', pos=[x, -1, 3], dir=[-x, 1, -2])
for y in [-2, 2]:
  arena.worldbody.add('light', pos=[-1, y, 3], dir=[1, -y, -2], attenuation=[3, 0, 0], castshadow=False)
arena.worldbody.add('camera', name='camera_1', pos=[-1, -1, 1], euler=[1.5, -0.5, 0])
arena.worldbody.add('camera', name='camera_2', pos=[-1, -1, 0.3], euler=[1.55, 2, 0])

# Add freejoint to ball so it can move freely
attachment_frame.add('joint', name='ball_joint', type='free', armature='0.00000175')

# Extract ball elements to read attributes
ball_body = ball_model.find('body', 'ball_body')
ball_geom = ball_model.find('geom', 'ball_geom')

In [6]:
from multifingered_ik import LevenbergMarquardtIK

# Some util functions as suggestions, you may or may not use them
# def generate_ball_contact_points():
#     """
#     Generates contact points for each finger on the surface of the ball
#     """
#     #YOUR CODE HERE

#     pass

def calculate_ball_contact_normals(contact_points: np.array, ball_center: np.array):
    """
    Input: array of contact points
    Return: array of normal vectors on the ball for each contact
    """
    #YOUR CODE HERE
    norms = contact_points - ball_center
    norms = norms / np.linalg.norm(norms)
    return norms

def set_camera_position(physics, camera_name, camera_position):
    camera_id = physics.model.name2id(camera_name, "camera")
    physics.named.model.cam_pos[camera_name] = camera_position
    return camera_id

def set_initial_configuration(physics):
    initial_qpos = [0, 0, 0, 0, 0, 0, 0, 0, -0.8, 0, 2, 0, -1.2, 3.2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    physics.data.qpos[:] = initial_qpos

In [7]:
# Define ball attributes
ball_radius = ball_geom.size[0]
ball_center = np.array(ball_body.pos)

physics = mjcf.Physics.from_mjcf_model(arena)

# Init scene
camera_id = set_camera_position(physics, "camera_1", [0.3, -0.5, 0.1])
camera_id2 = set_camera_position(physics, "camera_2", [1.7, 0.7, 0.3])
set_initial_configuration(physics)

physics.forward()
#PIL.Image.fromarray(physics.render(height=480, width=640, camera_id=camera_id2))

In [8]:
# # Here you can test your IK solver
# #Use IK to get the joint states of your target positions.
import mujoco

body_names = np.array(['sawyer/allegro_right/palm', 'sawyer/allegro_right/ff_tip_rubber', 'sawyer/allegro_right/mf_tip_rubber', 'sawyer/allegro_right/rf_tip_rubber', 'sawyer/allegro_right/th_tip_rubber'])

body_positions = np.array([[ 0.674, -0.185, 0.524], [ 0.723, -0.113, 0.56 ],[ 0.723, -0.118,0.516], [ 0.736, -0.113, 0.469], [ 0.601, -0.143, 0.67 ]]).T

body_orientations = np.array([[-0.626, -0.589, -0.392, -0.329], [ 0.33 , 0.305, -0.676, -0.584],[ 0.347, 0.286, -0.652, -0.61 ], [ 0.017, -0.071, -0.724, -0.686], [-0.967, 0.061, 0.029, 0.245]]).T

# model = physics.model
# data =physics.data
# body_ids = [physics.model.name2id(name, 'body') for name in body_names]
# jacp = np.zeros((3, model.nv)) #translation jacobian
# jacr = np.zeros((3, model.nv)) #rotational jacobian
# goal = [0.49, 0.13, 0.59]
# step_size = 0.5
# tol = 0.01
# alpha = 0.5
# damping = 0.15

# ik = LevenbergMarquardtIK(model, data, step_size, tol, alpha, jacp, jacr, damping, 100, physics)

# #Get desire point
# physics.reset()  # Use keyframes for initialization
# physics.data.qpos[:] = physics.data.qpos.copy()

# #reset qpos to initial value
# ik.calculate(body_positions, body_orientations, body_ids) #calculate the qpos

# result = data.qpos.copy()

# #Plot results

# physics.forward()
# #PIL.Image.fromarray(physics.render(height=480, width=640, camera_id=camera_id2))

#Back to initial config above ball
physics.reset()
set_initial_configuration(physics)
physics.forward()


In [ ]:
from AllegroHandEnv import AllegroHandEnvSphere
from grasp_synthesis import synthesize_grasp

# Run grasp synthesis algorithm to generate force closure hand onfiguration

palm_pos = ball_center + np.array([0.3, 0, ball_radius + 0.1])
# body_positions = generate_ball_contact_points()
# body_orientations = np.array([[0,0,0,0] for _ in range(5)]).T

ik = LevenbergMarquardtIK(
    model=physics.model,
    data=physics.data,
    step_size=0.5,
    tol=0.01,
    alpha=0.3,
    jacp=np.zeros((3, physics.model.nv)),  # Translation Jacobian
    jacr=np.zeros((3, physics.model.nv)),  # Rotational Jacobian
    damping=0.1,
    max_steps=100,
    physics=physics
)


# physics.data.qpos[:] = physics.data.qpos.copy()
# physics.reset()
# ik.calculate(body_positions, body_orientations, body_ids)
# physics.data.qpos[8] += 0.27
# physics.data.qpos[10] += -0.2
# physics.forward()
# body_ids = [physics.model.name2id(name, 'body') for name in body_names]
# fingertip_names = np.array(['sawyer/allegro_right/ff_tip_rubber', 'sawyer/allegro_right/mf_tip_rubber', 'sawyer/allegro_right/rf_tip_rubber', 'sawyer/allegro_right/th_tip_rubber'])
# ballenv = AllegroHandEnvSphere(physics, ball_center, ball_radius, slice(16, 30), "ball")
# grasp = synthesize_grasp(ballenv, physics.data.qpos[ballenv.q_h_slice].copy(), fingertip_names)
# physics.data.qpos[8] += -0.3




#bideo
options = mj.MjvOption()
mj.mjv_defaultOption(options)
options.flags[mj.mjtVisFlag.mjVIS_CONTACTPOINT] = True
options.flags[mj.mjtVisFlag.mjVIS_CONTACTFORCE] = False
options.flags[mj.mjtVisFlag.mjVIS_TRANSPARENT] = True

# tweak scales of contact visualization elements
physics.model.ptr.vis.scale.contactwidth = 0.1
physics.model.ptr.vis.scale.contactheight = 0.03
physics.model.ptr.vis.scale.forcewidth = 0.05
physics.model.ptr.vis.map.force = 0.05

duration = 5
framerate = 30
dt = 1 / (duration * framerate)
video = []
n_steps = 200

# allocate
sim_time = np.zeros(n_steps)
ncon = np.zeros(n_steps)
force = np.zeros((n_steps,3))
forcetorque = np.zeros(6)


#Bring arm down to the ball
physics.reset()
set_initial_configuration(physics)
physics.data.qpos[8] += 0.27
physics.data.qpos[10] += -0.2

#Grasp the ball
body_ids = [physics.model.name2id(name, 'body') for name in body_names]
fingertip_names = np.array(['sawyer/allegro_right/ff_tip_rubber', 'sawyer/allegro_right/mf_tip_rubber', 'sawyer/allegro_right/rf_tip_rubber', 'sawyer/allegro_right/th_tip_rubber'])

ballenv = AllegroHandEnvSphere(physics, ball_center, ball_radius, slice(14, 30), "ball")

grasp = synthesize_grasp(ballenv, physics.data.qpos[ballenv.q_h_slice].copy(), fingertip_names)
physics.data.qpos[ballenv.q_h_slice] = grasp
physics.forward()

#Get current palm position
palm_id = physics.model.name2id('sawyer/allegro_right/palm', 'body')
palm_pos = physics.data.xpos[palm_id].copy()
lifted_palm_pos = palm_pos + np.array([0, 0, 0.1])  # lift by 10cm

# Get current fingertip positions
fingertip_ids = [physics.model.name2id(name, 'body') for name in fingertip_names]
fingertip_positions = [physics.data.xpos[i].copy() for i in fingertip_ids]

# Compute relative offsets from palm to fingertips
offsets = [tip - palm_pos for tip in fingertip_positions]

# Compute new fingertip positions (maintain grasp geometry)
lifted_fingertip_positions = [lifted_palm_pos + offset for offset in offsets]

# Stack palm and fingertips for IK
target_positions = np.vstack([lifted_palm_pos, lifted_fingertip_positions])

# Orientations: keep as zeros or use current orientations if needed
target_orientations = np.zeros((4, target_positions.shape[0]))  # shape (4, n_bodies)

# Body IDs: palm + fingertips
body_ids = [palm_id] + fingertip_ids

# Run IK to move the arm so the palm/fingertips reach the new positions
# ik.calculate(target_positions.T, target_orientations, body_ids)
# physics.forward()



# with mj.Renderer(physics.model.ptr, 480, 640) as renderer:
#     for i in range(n_steps):
#         while physics.data.time * framerate < i:
#             # Control robot and collect data for plotting
#             #YOUR CODE HERE

            
            
#             physics.step()
#         # Render scene, feel free to change camera
#         renderer.update_scene(physics.data.ptr, "camera_2", options)
#         frame = renderer.render()
#         video.append(frame)

video = []
n_steps = 100
lift_height = 0.15  # total lift in meters

#Lift the ball

with mj.Renderer(physics.model.ptr, 480, 640) as renderer:
    for i in range(n_steps):
        # a. Compute lifted palm position for this frame
        dz = lift_height * (i / (n_steps - 1))
        lifted_palm_pos = palm_pos + np.array([0, 0, dz])
        lifted_fingertip_positions = [lifted_palm_pos + offset for offset in offsets]
        target_positions = np.vstack([lifted_palm_pos, lifted_fingertip_positions])
        target_orientations = np.zeros((4, target_positions.shape[0]))  # or use current orientations

        # b. Run IK to move the arm (hand joints fixed)
        physics.data.qpos[ballenv.q_h_slice] = grasp  # keep hand fixed
        ik.calculate(target_positions, target_orientations.T, [palm_id] + fingertip_ids)
        physics.forward()

        # c. Render and save frame
        renderer.update_scene(physics.data.ptr, "camera_2", options)
        frame = renderer.render()
        video.append(frame)

media.show_video(video, fps=30)
#PIL.Image.fromarray(physics.render(height=480, width=640, camera_id=camera_id2))
# PIL.Image.fromarray(physics.render(height=480, width=640, camera_id=camera_id))

This browser does not support the video tag.


In [ ]:
#Grasp ball

# body_ids = [physics.model.name2id(name, 'body') for name in body_names]
# fingertip_names = np.array(['sawyer/allegro_right/ff_tip_rubber', 'sawyer/allegro_right/mf_tip_rubber', 'sawyer/allegro_right/rf_tip_rubber', 'sawyer/allegro_right/th_tip_rubber'])

# ballenv = AllegroHandEnvSphere(physics, ball_center, ball_radius, slice(16, 30), "ball")

# grasp = synthesize_grasp(ballenv, physics.data.qpos[ballenv.q_h_slice].copy(), fingertip_names)
# physics.data.qpos[8] += -0.3
# physics.forward()

# PIL.Image.fromarray(physics.render(height=480, width=640, camera_id=camera_id2))

In [ ]:
#Lift ball
# physics.data.qpos[8] += -0.3
# physics.forward()
# PIL.Image.fromarray(physics.render(height=480, width=640, camera_id=camera_id2))

In [ ]:
# Here are some flags you can set to visualize contact forces
# We suggest that you visualize the results of your grasp synthesis algorithm when debugging
# Visualize contact frames and forces, make body transparent
options = mj.MjvOption()
mj.mjv_defaultOption(options)
options.flags[mj.mjtVisFlag.mjVIS_CONTACTPOINT] = False
options.flags[mj.mjtVisFlag.mjVIS_CONTACTFORCE] = False
options.flags[mj.mjtVisFlag.mjVIS_TRANSPARENT] = False

# tweak scales of contact visualization elements
physics.model.ptr.vis.scale.contactwidth = 0.1
physics.model.ptr.vis.scale.contactheight = 0.03
physics.model.ptr.vis.scale.forcewidth = 0.05
physics.model.ptr.vis.map.force = 0.05

In [ ]:
# Here's some skeleton code to run a control loop in MuJoCo
# This is just an initial suggestion, so feel free to change anything about it

duration = 5
framerate = 30
dt = 1 / (duration * framerate)
video = []
n_steps = 200

# allocate
sim_time = np.zeros(n_steps)
ncon = np.zeros(n_steps)
force = np.zeros((n_steps,3))
forcetorque = np.zeros(6)

physics.reset()

# Initialize state
#YOUR CODE HERE

# Run control loop
with mj.Renderer(physics.model.ptr, 480, 640) as renderer:
    for i in range(n_steps):
        while physics.data.time * framerate < i:
            # Control robot and collect data for plotting
            #YOUR CODE HERE

            physics.step()
        # Render scene, feel free to change camera
        renderer.update_scene(physics.data.ptr, "camera_2", options)
        frame = renderer.render()
        video.append(frame)

# Plot
# This is just to serve as an example for plotting. You'll likely have to make some changes for the deliverables
_, ax = plt.subplots(3, 1, sharex=True, figsize=(10, 10))

lines = ax[0].plot(sim_time, force)
ax[0].set_title('Contact force')
ax[0].set_ylabel('Newton')
ax[0].legend(list(lines), ('normal z', 'friction x', 'friction y'))
ax[1].plot(sim_time, ncon)
ax[1].set_title('Number of contacts')
ax[1].set_yticks(range(10))

ax[2].plot(sim_time, force[:,0])
ax[2].set_yscale('log')
ax[2].set_title('Normal (z) force - log scale')
ax[2].set_ylabel('Newton')


In [ ]:
# Display video
media.show_video(video, fps=30)